# Read, modify, and extract data from network and catchment Res1D files

With mikeio1d it is possible:
* Read res1d file data into pandas data frame
* Modify the res1d file contents using data frame
* Extract res1d time series data to `csv`, `dfs0`, or `txt` files

Currently there are the following types of res1d files:
* Network
* Catchment
* Long Term Statistics (events), see [res1d_lts.ipynb](res1d_lts.ipynb)
* Long Term Statistics (chronological), see [res1d_lts.ipynb](res1d_lts.ipynb)
* Simple statistics (not fully supported by mikeio1d)

In this notebook we will give examples how to work with network and catchment res1d files.

In [ ]:
from mikeio1d import Res1D

## Network result file

Load network result file into mikeio1d Res1D object and print basic information about the file:

In [ ]:
file_path_network = '../tests/testdata/network.res1d'
res1d_network = Res1D(file_path_network)
res1d_network.info()

Let's read the entire network res1d file time series data into a data frame and show just the first time steps:

In [ ]:
df_network = res1d_network.read_all()
df_network.head()

To retrieve water level in the link **l100l1** (the upstream **chainage=0** and the downstream **chainage=47.6827**) can be retrieved and plotted as:

In [ ]:
df_network[['WaterLevel:100l1:0','WaterLevel:100l1:47.6827']]

In [ ]:
df_network[['WaterLevel:100l1:0','WaterLevel:100l1:47.6827']].plot()

## Catchment result file

Load network result file into mikeio1d Res1D object and print basic information about the file:

In [ ]:
file_path_catchments = '../tests/testdata/catchments.res1d'
res1d_catchments = Res1D(file_path_catchments)
res1d_catchments.info()

Let's read the entire network res1d file time series data into a data frame and show just the first time steps:

In [ ]:
df_catchments = res1d_catchments.read_all()
df_catchments.head()

## Access quantity and network/catchment information

List quantities found in a network and catchment res1d files as a list of strings:

In [ ]:
res1d_network.quantities

In [ ]:
res1d_catchments.quantities

List of nodes/reaches/catchments and corresponding quantities on nodes/reaches/catchments in a file can be obtained using the below properties:

In [ ]:
res1d_network.nodes

In [ ]:
res1d_network.reaches

In [ ]:
res1d_catchments.catchments

## Query specific time series using query classes

In mikeio1d it is possible to load specific time series data using queries. Currently there are the following query classes:
* `QueryDataCatchment`
* `QueryDataNode`
* `QueryDataReach`
* `QueryDataStructure`
* `QueryDataGlobal`

In [ ]:
from mikeio1d.res1d import QueryDataNode, QueryDataReach, QueryDataCatchment

In [ ]:
# Read a specific reach
query1 = QueryDataNode('WaterLevel', '1')

# Read a specific reach and chainage
query2 = QueryDataReach('WaterLevel', '104l1', 34.4131)

# Combine queries to extract in one go
queries_network = [query1, query2]

In [ ]:
df_network_query = res1d_network.read(queries_network)
df_network_query.plot()

In [ ]:
# Read a specific catchment
query1 = QueryDataCatchment('TotalRunOff', '100_16_16')

# Read another specific catchment
query2 = QueryDataCatchment('TotalRunOff', '105_1_1')

# Combine queries to extract in one go
queries_catchments = [query1, query2]

In [ ]:
df_catchments_query = res1d_catchments.read(queries_catchments)
df_catchments_query.plot()

## Querying specific time series using auto-completion

It is possible to query data with auto-completion using the following properties of the Res1D object:
* `catchments`
* `nodes`
* `reaches`
* `structures`
* `global_data`

The variable name generated for auto-completion will have a particular string appended if the location id starts with a number or other special character not supported in Python variable names.
The appended strings are:
* catchments : `c_`
* nodes: `n_`
* reaches: `r_`
* structures: `s_`
* global data: `g_`
* chainages: `m_`

Here is an example to query water level on the reach **100l1** (chainages **m=0**, **m=47.6827**) and node **100**

In [ ]:
# Assign aliases for reaches and nodes
reaches = res1d_network.reaches
nodes = res1d_network.nodes

# Add to to current active queries
reaches.r_100l1.m_0.WaterLevel.add()
reaches.r_100l1.m_47_6827.WaterLevel.add()
nodes.n_100.WaterLevel.add()

# Read the current active query
df_network_query = res1d_network.read()
df_network_query.plot()

In [ ]:
# Assign alias for catchments
catchments = res1d_catchments.catchments

# Add to to current active queries
catchments.c_100_16_16.TotalRunOff.add()
catchments.c_105_1_1.TotalRunOff.add()

# Read the current active query
df_catchments_query = res1d_catchments.read()
df_catchments_query.plot()

## Filtered reading of the result files

It is possible to load a res1d file in a filtered way, that only specified locations are loaded into memory. Let's load nodes **1**, **2**, **3**, and reach **99l1**.

In [ ]:
file_path_network = '../tests/testdata/network.res1d'
res1d_network_filtered = Res1D(file_path_network, nodes=['1', '2', '3'], reaches=['99l1'])
df_network_filtered = res1d_network_filtered.read_all()
df_network_filtered

Similarly for catchments. Let's load data for catchment **100_16_16**.

In [ ]:
file_path_catchments = '../tests/testdata/catchments.res1d'
res1d_catchments_filtered = Res1D(file_path_catchments, catchments=['100_16_16'])
df_catchments_filtered = res1d_catchments_filtered.read_all()
df_catchments_filtered

## Modifying a res1d file

Here we will show how to modify a res1d file. Such functionality is useful for modifications of a hotstart file.

In [ ]:
file_path_network = '../tests/testdata/network.res1d'
res1d_network_mod = Res1D(file_path_network)

Let's load all node water level time series:

In [ ]:
res1d_network_mod.nodes.WaterLevel.add()
df_network_mod = res1d_network_mod.read(column_mode='all')
print('Current maximum water level: ', df_network_mod.max().max())

Multiply the water level data by a factor of 2 and write it to a new file

In [ ]:
df_network_mod = df_network_mod.multiply(2.0)
file_path_new = file_path_network.replace('network.res1d', 'NetworkFactorTwo.res1d')
res1d_network_mod.modify(df_network_mod, file_path=file_path_new)

Load the data again into a data frame

In [ ]:
res1d_network_mod.nodes.WaterLevel.add()
df_network_mod = res1d_network_mod.read()
print('Current maximum water level: ', df_network_mod.max().max())

Load the newly written file

In [ ]:
print('New file path:', file_path_new)
res1d_network_new = Res1D(file_path_new)
res1d_network_new.nodes.WaterLevel.add()
df_network_new = res1d_network_new.read()
print('Current maximum water level:', df_network_new.max().max())

## Extracting time series data to dfs0 file

It is possible to extract the res1d time series data to a dfs0 file. The idea is again to create queries, which are used to pick which time series to extract. Let's extract all node water level:

In [ ]:
res1d_network.nodes.WaterLevel.add()
file_path_dfs0 = file_path_network.replace('network.res1d', 'NetworkNodeWaterLevel.dfs0')
res1d_network.to_dfs0(file_path=file_path_dfs0)

## Extracting reach start, end, or sum values as numpy arrays

It is possbile to get time series values as numpy arrays at reach start or end chainage without specifying chainage number

In [ ]:
values_start = res1d_network.get_reach_start_values("9l1", "WaterLevel")
values_end = res1d_network.get_reach_end_values("9l1", "WaterLevel") 

Get time series values summed for all gridpoints in reach with given quantity, i.e. useful for getting total volume in reach. 

In [ ]:
values_sum = res1d_network.get_reach_sum_values("9l1", "Discharge")

Create pandas time series from `values_sum` and plot

In [ ]:
import pandas as pd
pd.Series(values_sum, index = res1d_network.time_index).plot(ylabel="9l1: " + "Discharge")